## Infrence on dataset for panoptic

In [1]:
from detectron2.utils.logger import setup_logger
from detectron2.data import MetadataCatalog, DatasetCatalog
setup_logger()

train_json = "./annotations/train_panoptic.json"
image_root = "/home/wenisch/Atom360/AI/Learning/data/dataset/images"

logger = setup_logger(name=__name__)

# # COCO classes
# CLASSES = [ #'N/A', 
#     'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
#     'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
#     'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
#     'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
#     'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
#     'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
#     'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
#     'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
#     'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
#     'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
#     'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
#     'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
#     'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
#     'toothbrush'
# ]

CLASSES = []

/home/wenisch/anaconda3/envs/detectron/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from detectron2.data.datasets import register_coco_instances

# MetadataCatalog.get("dataset_test").set(thing_classes= CLASSES)
meta_test = MetadataCatalog.get("dataset_test")
print("Meta test is :"+ str(meta_test.as_dict()))
register_coco_instances("dataset_test", meta_test.as_dict(), train_json, image_root)

Meta test is :{'name': 'dataset_test', 'thing_classes': []}


In [3]:
import os, json, cv2, random
from detectron2.utils.visualizer import Visualizer
from matplotlib import pyplot as plt

dataset_dicts_val = DatasetCatalog.get("dataset_test")
noneobjects = 0

for d in random.sample(dataset_dicts_val, 3):
    img = cv2.imread(d["file_name"])
    if img is not None:
        visualizer = Visualizer(img[:, :, ::-1], metadata=meta_test, scale=1)
        out = visualizer.draw_dataset_dict(d)
        #cv2_imshow(out.get_image()[:, :, ::-1])
        plt.imshow(out.get_image())
    else:
        noneobjects += 1
        print("File missing"+ d["file_name"])
print("Number of Missing files ", noneobjects)

ValueError: min() arg is an empty sequence

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
for d in random.sample(dataset_dicts_val, 3):
    im = cv2.imread(d["file_name"])
    # outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
    ids = [info['category_id'] for info in segments_info]
    classes = [meta_test.thing_classes[id] for id in ids]
    print(classes)
    plt.figure(figsize = (20,20))
    plt.imshow(out.get_image())

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("dataset_test", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "dataset_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`